# ToDo: 
- criticality of intersections. should all of them be nodes?
- try building network without OSM data [done]
- add coordinates to nodes to plot the network?
- try building network with OSM data (first,requires I add width to OSM data)
- create network of all roads, with weights
- investigate algorithm for filling up links randomly with weight threshold

# Road network of Milan

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import contextily as cx
import matplotlib.pyplot as plt
import networkx as nx

In [ ]:
vehicle_path = "C:/Users/rickb/Documents/scuola/THESIS/datasets/Milan/DBT_2020/SHAPE/AC_VEI_AC_VEI_SUP_SR.shp"
gdf = gpd.read_file(vehicle_path)

In [ ]:
gdf.drop(['AC_VEI_FON', 'AC_VEI_LIV', 'AC_VEI_SED', 'CLASSREF'],axis = 1, inplace = True)
gdf.rename(columns={'SUBREGID':'ID', 'NOME': 'NAME', 'AC_VEI_ZON': 'TYPE'}, inplace = True)

In [ ]:
gdf_tot = gdf.copy()

In [ ]:
pattern1 = ('01','02')
 # portions of road (e.g not intersections or parking lots) start with 01 in TYPE
 # intersections, squares, and roundabouts start with 02 in TYPE
gdf_tot = gdf_tot[~gdf_tot['NAME'].str.contains('TANGENZIALE', regex = False)] #removing tangenziali
gdf_tot = gdf_tot[gdf_tot.TYPE.str.startswith(pattern1)]

## Areas and perimeters of streets
We now try to plot areas and perimeters of streets, with and without using the simplify method to see if there are significant differences between the two.
our variables are gdf_roads, gdf_piaz, gdf_round, or all together in gdf_roads_piaz.


NB for the moment we're excluding tangenziali because some of the blocks are very large and make visualization difficult

In [ ]:
OSM_crs = 3857
gdf_tot.to_crs(epsg=OSM_crs, inplace = True)

gdf_tot['Perimeter'] = gdf_tot.length
gdf_tot['Area'] = gdf_tot.area

### Average width calculation: 
Area is length times width for rectangles
Perimeter is 2(length) + 2(width)
$A = lw$
$P = 2l+2w$
brings us to solve for width as   

$P = 2\frac{A}{w}+2w$ 
so  
$w^2 -\frac{P}{2}w+A = 0$

In [ ]:
gdf_tot['temp'] = gdf_tot.Area/gdf_tot.Area # create column of ones
gdf_tot['SemiPeri'] = -gdf_tot.Perimeter/2 # i need it negative for the equation

def calculate_roots(row):
    coefficients = row[['temp', 'SemiPeri', 'Area']].values
    roots = np.roots(coefficients).real
    return roots

#gdf_tot['roots'] = gdf_tot.apply(calculate_roots, axis=1)
gdf_tot['roots'] = gdf_tot[['temp', 'SemiPeri', 'Area']].apply(calculate_roots, axis=1)
gdf_tot[['root1', 'root2']] = pd.DataFrame(gdf_tot['roots'].tolist(), index=gdf_tot.index)
gdf_tot['width'] = gdf_tot['root2']
gdf_tot = gdf_tot.drop(['Perimeter', 'Area', 'temp', 'SemiPeri', 'roots', 'root2'], axis = 1)

NB with this method root2 is always the width because all segments of road are longer than they are wide, and root2 is always the smaller of the two roots, by construction of the method. at most there can be entries where root1=root2, which would mean that they both equal the width.

## dividing by zones with sjoin
let's try and get a better division into zones, to have more in depth plots

In [ ]:
from geopandas.tools import sjoin

In [ ]:
administrative_path = "C:/Users/rickb/Documents/scuola/THESIS/datasets/Milan/DBT_2020_new/DBT 2020 - SHAPE/Municipi.shp"
gdf2 = gpd.read_file(administrative_path)
gdf2 = gdf2.to_crs(epsg = OSM_crs)

gdf_zone_tot = gdf_tot.sjoin(gdf2, how = 'inner',predicate = 'intersects') # requires gpd > 0.9
#gdf_zone = gpd.sjoin(gdf_roads_piaz, gdf2, how = 'inner', op = 'intersects') #is equivalent, with older syntax
gdf_zone_tot = gdf_zone_tot.drop(['AREA', 'PERIMETRO', 'index_right'], axis =1)

In [ ]:
neighborhood_path = "C:/Users/rickb/Documents/scuola/THESIS/datasets/Milan/Quartieri milano_real/NIL_WM.shp"
gdf_N = gpd.read_file(neighborhood_path)
gdf_N = gdf_N.to_crs(epsg = OSM_crs)
gdf_N = gdf_N.drop(['Valido_dal', 'Fonte', 'Shape_Leng', 'Shape_Area', 'OBJECTID', 'Valido_al'] ,axis=1)


gdf_N_tot = gdf_zone_tot.sjoin(gdf_N, how = 'inner',predicate = 'intersects')
gdf_N_tot = gdf_N_tot.drop(['index_right'], axis = 1)


Finally, let's split our main dataframe into two: one with only roads, and one with only intersections.

In [ ]:
pattern2 = ('01')
pattern3 = ('02') 
gdf_no_int = gdf_N_tot[gdf_tot.TYPE.str.startswith(pattern2)]
gdf_int = gdf_N_tot[gdf_tot.TYPE.str.startswith(pattern3)]

In [ ]:
#Now isolate an example neighborhood
gdf_N_Stadera = gdf_N_tot[gdf_N_tot['NIL'] == 'STADERA - CHIESA ROSSA - Q.RE TORRETTA - CONCA FALLATA']
gdf_int_Stadera = gdf_int[gdf_int['NIL'] == 'STADERA - CHIESA ROSSA - Q.RE TORRETTA - CONCA FALLATA']
gdf_no_int_Stadera = gdf_no_int[gdf_no_int['NIL'] == 'STADERA - CHIESA ROSSA - Q.RE TORRETTA - CONCA FALLATA']
#and a single street in that neighborhood
gdf_N_Volv = gdf_N_Stadera[gdf_N_Stadera['NAME'] == 'VIA VOLVINIO']
gdf_no_int_Volv = gdf_no_int[gdf_no_int['NAME'] == 'VIA VOLVINIO']
gdf_int_Volv = gdf_int[gdf_int['NAME'] == 'VIA VOLVINIO']


Here's an example plot of a neighborhood:

In [ ]:
fig, ax = plt.subplots(1,1, figsize = (8,8))
gdf_N_Stadera.plot(ax = ax, alpha = 0.5)
cx.add_basemap(ax, crs=gdf_N.crs, zoom = 15, source=cx.providers.CartoDB.Positron) #providers.Esri.WorldImagery for satellite
#source=cx.providers.CartoDB.Positron)
plt.title("The Stadera Neighborhood of Milan")
plt.show()

## Creating custom ranges by distance

Here's a function to create a Geodataframe with all roads within a given distance from the road given as input

In [ ]:
def within_dist(street, dist, gdf):
    #function creates geodataframe with all streets of gdf within distance dist (in meters) of street.
    #street is a geodataframe, dist is a positive number, and gdf is the geodataframe dataset.
    temp = street.copy()
    temp.geometry = temp.geometry.buffer(dist)
    temp = temp.filter(['geometry']) #so sjoin doesn't give suffixes and i don't have to rename later
    gdf_distanced = gdf.sjoin(temp, how='inner', predicate='intersects')
    gdf_distanced = gdf_distanced.dropna()
    gdf_distanced = gdf_distanced.drop_duplicates(subset=['width'], keep='first') #removes streets within 2 buffers of a polygon
    gdf_distanced = gdf_distanced.iloc[:,:-1] #drops index_R column
    return gdf_distanced

In [ ]:
M = 100
gdf_test = within_dist(gdf_N_Volv, M, gdf_tot)


In [ ]:
fig, ax = plt.subplots(1,1, figsize = (8,8))

gdf_test.plot(ax = ax, alpha = 0.5)
cx.add_basemap(ax, crs=gdf_N.crs, zoom = 15, source=cx.providers.CartoDB.Positron) #providers.Esri.WorldImagery for satellite
plt.title(f"roads within {M} meters from Via Volvinio")
plt.show()

Let's try to plot roads color coding by width

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(8,8))
gdf_test.plot(ax = ax, cmap = 'viridis', column = 'width', legend = True, vmin = 1, vmax = 30 )
cx.add_basemap(ax, crs=gdf_test.crs, source=cx.providers.Esri.WorldImagery, alpha =0.3) #providers.Esri.WorldImagery for satellite
plt.title(f"roads within {M} meters from Via Volvinio \n color coded by width")
plt.show()

# The Network
It should be pretty straightforward: make intersections the nodes, and make roads the edges. road width are the weights. 
However, there is a problem with what exactly it means to be an intersection.
NB for now we will consider all streets in the manner which is most convenient, i.e as two-way streets, unless otherwise specified.
### Examining intersections
Let's examine the case of a relatively simple street, Via Volvinio



In [ ]:
fig, ax = plt.subplots(1,1, figsize = (6,6))

gdf_no_int_Volv.plot(ax = ax, alpha = 0.5)
cx.add_basemap(ax, crs=gdf_N.crs, zoom = 17, source=cx.providers.CartoDB.Positron) #providers.Esri.WorldImagery for satellite
plt.show()

The upper part of the street divides in two, because of a barrier, then there is a blank space (considered an intersection), even though the road continues onwards without being intersected. how should this be considered? a new node seems excessive, but then what can we do? maybe after the fact i can say that that node must be eliminated since it connects to only two roads, one of which with a double connection?

In [ ]:
fig, ax = plt.subplots(1,1, figsize = (6,6))

gdf_N_Volv.plot(ax = ax, column = 'TYPE', alpha = 0.5)
cx.add_basemap(ax, crs=gdf_N.crs, zoom = 17, source=cx.providers.CartoDB.Positron) #providers.Esri.WorldImagery for satellite
plt.show()

According to our pdf document, 0205 corresponds to "incrocio". I guess it's ok to consider it as such.

## Actually trying to create the network

remember, my network has:
- intersections as nodes.
- the actual roads as edges.
- their width and name will be weights/edge attributes.


To find the edges of the network, we take an intersection and use the within_dist function with a distance = 1 to find
all roads that are immediately adjacent to the intersection. 
These are the "stubs" of our graph, i.e lines that connect to a node and nothing else.  
When we do this for all intersection, the edges of the network will simply be the common stubs between pairs of nodes.  
Our road network will be a MultiGraph, because some intersections may be connected by two or more different roads.

In [ ]:
#we define a variation of the within_dist function. This one keeps duplicate entries because they are useful for finding stubs.

def within_dist_dupes(street, dist, gdf):
    #function creates geodataframe with all streets of gdf within distance dist (in meters) of street.
    #street is a geodataframe, dist is a positive number, and gdf is the geodataframe dataset.
    temp = street.copy()
    temp.geometry = temp.geometry.buffer(dist)
    temp = temp.filter(['geometry']) #so sjoin doesn't give suffixes and i don't have to rename later
    gdf_distanced = gdf.sjoin(temp, how='inner', predicate='intersects')
    gdf_distanced = gdf_distanced.dropna()
    return gdf_distanced

This function is only called on dataframes with indexes from 0 to N.  
When we sjoin an intersection with its surrounding streets, our resulting dataframe will tell us, in each row, the index of the intersection
that the given road is adjacent to. this information will be contained in "index_right", which is the new name given to what was the intersection index in the original dataframe.  

This means each row of our sjoined dataframe represents a stub of the node "index_right".  


We divide into N dataframes based on index_right, to have a dataframe of the adjacent edges to one road (i.e the stubs).   
there is a connection between dataframe_i and dataframe_j for each common stub between the two.  

Finally, the function that performs these operations and finds the connections is the following:

In [ ]:
def make_edges(gdf_tot):
    #takes dataset with roads and intersections, creates edgelist of nodes with weights of edges
    pattern1, pattern2 = '01', '02'
    no_ints = gdf_tot[gdf_tot.TYPE.str.startswith(pattern1)]
    ints = gdf_tot[gdf_tot.TYPE.str.startswith(pattern2)]
    #we need indices from 0 --> reset
    ints.reset_index(inplace = True, drop = True)
    no_ints.reset_index(inplace = True, drop = True)
    stubs = within_dist_dupes(ints, 1, no_ints) #all stubs, i.e all roads connected to all nodes
    grouped = stubs.groupby('index_right') #one dataframe for each node
    edges = {} # will contain intersections of each node 
    edge_list = pd.DataFrame(columns = ['from','to','weight'])
    for node, group in grouped:
        stubs = stubs[stubs['index_right'] != node] #removing "self" from gdf that we will merge onto, to avoid self connections. also removes redundancies  
        edges[node] = pd.merge(group,stubs, on = 'ID', how = 'inner')
        edge_list_temp = pd.DataFrame({'from': edges[node].index_right_x, 'to': edges[node].index_right_y, 'weight': edges[node].width_x})
        edge_list = pd.concat([edge_list,edge_list_temp])
    return edge_list
#now edges should be a dictionary where each key will have only the nodes it is connected to as values.
#the final step would be to make a list where each key is 

In [ ]:
edge_list_t = make_edges(gdf_N_Stadera)
G = nx.from_pandas_edgelist(edge_list_t, 'from', 'to', edge_attr=["weight"] , create_using=nx.MultiGraph())

In [ ]:
edge_list_t.head(-10) 

The first two nodes have several connections to a single other node because they are kind of strange, peripheral roads.  
I don't think this is a problem in general, considering most roads are well behaved.

In [ ]:
temp = gdf_N_Stadera[gdf_N_Stadera.NAME == 'VIA DEL MARE']
t = within_dist(temp, 5, gdf_N_Stadera)
fig, ax = plt.subplots(1,1, figsize = (6,6))
t.plot(ax = ax, column = 'NAME', alpha = 0.5)
cx.add_basemap(ax, crs=gdf_N.crs, zoom = 16, source=cx.providers.CartoDB.Positron) #providers.Esri.WorldImagery for satellite
plt.title('roads with multiple connections between each other')
plt.show()

In [ ]:
pos = nx.random_layout(G)
nx.draw_networkx_nodes(G, pos, node_color = 'r', node_size = 100, alpha = 1)
ax = plt.gca()
for e in G.edges:
    ax.annotate("",
                xy=pos[e[0]], xycoords='data',
                xytext=pos[e[1]], textcoords='data',
                arrowprops=dict(arrowstyle="-", color="0.5",
                                shrinkA=5, shrinkB=5,
                                patchA=None, patchB=None,
                                connectionstyle="arc3,rad=rrr".replace('rrr',str(0.3*e[2])
                                ),
                                ),
                )
plt.axis('off')
plt.show()

# Network with OSMnx package

to be completed